# Chapter 8 Lab/Assignment (20 points)
## Polynomials, Qualitative variables, and interactions

> <u>INSTRUCTIONS:</u> 
> 
> Complete the problems listed below.
> 
> <ul>
  <li>The assignment may be completed in this notebook or in a separate document</li>
  <li>Homework must be submitted individually</li>
  <li>To receive full credit, include relevant R code/output and answer all questions</li>
  <li>Upload your assignment to Canvas under Assignments > Homework #.</li>
  <li>Use the following file naming convention:  Lastname_Firstname_HW# (but put your own name instead).</li>
  <li>The assignment must be submitted in .html or .pdf </li>
</ul>



### (10 points) Write and submit a first draft of your Exploratory Data Analysis section (part 4) of your class project.

- Please refer to the original project description on Canvas for details

### -------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Data Analysis Objective I

The goal of this lab assignment is to walk you through a multiple linear regression analysis using concepts from in Chapter 8.

We are interested in the relationship between the number of active physicians (Y) and income per capita ($X_{1}$) across the four regions ($X_{2}$) in the CDI dataset. Please refer to the CDI dataset for a thorough description of variables.

### Necessary packages

In [ ]:
#install.packages("GGally")
#install.packages("ggplot2")

library(ggplot2)
library(GGally)
library(car)
library(multcomp)
#library(stats)

### Load the CDI dataset

In [ ]:
options(repr.plot.width=5, repr.plot.height=5)

# read in data
cdi <- read.table('CDI.txt', header=TRUE)

In [ ]:
# view first few lines of data
head(cdi)

### Recode region

R currently considers $cdi\$region$ as a continuous variable. We must convert $cdi\$region$ to a factor (categorical/qualitative) variable

In [ ]:
# region is still considered a quantitative variable ('numeric' or 'integer')
class(cdi$region)

In [ ]:
# convert to a factor
cdi$region <- factor(cdi$region)
class(cdi$region)

In [ ]:
# old labels
levels(cdi$region)

In [ ]:
# new labels
levels(cdi$region) <- c("NE", "NC", "S", "W")
levels(cdi$region)

### Visualize relationships using ggpairs (or pairs)

<font color='red'>**(2 points) Question:**</font> 
- What functional relationships do you observe in the plots below for each region? Use the pairs plots and lowess smooths to answer the question.

In [ ]:
# without log transformation
options(repr.plot.width=8, repr.plot.height=5)
plot_frame <- data.frame("income_per_capita" = cdi$income_percapita, 
                         "number_active_physicians" = (cdi$number_active_physicians), 
                         "region" = cdi$region)
ggpairs(plot_frame, aes(color=region, alpha=100), binwidth=30)

In [ ]:
# with log transformation
options(repr.plot.width=8, repr.plot.height=5)
plot_frame <- data.frame("income_per_capita" = cdi$income_percapita, 
                         "log_number_active_physicians" = log(cdi$number_active_physicians), 
                         "region" = cdi$region)
ggpairs(plot_frame, aes(color=region, alpha=100), binwidth=30)

### Use scatterplots and lowess smooths to identify the functional relationships

In [ ]:
options(repr.plot.width=8, repr.plot.height=5)
ggplot(cdi, aes(x=income_percapita, y=log(number_active_physicians), color=region, alpha = 0.1)) +
  geom_point() + 
  geom_smooth(method=loess, aes(fill=region), alpha = 0.2)

Removing the points from the scatter plot can make it easier to visualize the trends, however, it is best to investigate both visualizations.

In [ ]:
options(repr.plot.width=8, repr.plot.height=5)
ggplot(cdi, aes(x=income_percapita, y=log(number_active_physicians), color=region, alpha = 0.1)) +
  #geom_point() + 
  geom_smooth(method=loess, aes(fill=region), alpha = 0.1)

### Fit regression models based on visual observations

- It appears the relationships are quadratic or even cubic; We can start with a cubic relationship
- We can allow statistical testing in our regressions to determine if these are significant trends

#### Fitting a cubic regression 

In [ ]:
# use I() to fit a polynomial
fit <- lm(log(number_active_physicians)~
          cdi$income_percapita + 
          I(cdi$income_percapita^2) + 
          I(cdi$income_percapita^3), data=cdi)
summary(fit)

options(repr.plot.width=10, repr.plot.height=3.5)
par(mfrow=c(1,3))
plot(fit, which=c(1,2,3))

<font color='red'>**(2 points) Question:**</font> 
- What do the F-test results in the summary suggest about this regression?
- Are any of the polynomial coefficients significant?
- What does this imply?

#### Orthogonal polynomials
We can use orthogonal polynomials using the $\texttt{poly()}$ function in R to avoid multicollinearity

In [ ]:
fit <- lm(log(number_active_physicians)~
          poly(income_percapita, 3)        # orthogonal polynomial
          , data=cdi)
summary(fit)

Great! Multicollinearity fixed!

In [ ]:
fit <- lm(log(number_active_physicians)~
          poly(income_percapita, 3) +        # orthogonal polynomial
          region +                           # 4 regions
          poly(income_percapita, 3):region   # interaction
          , data=cdi)
summary(fit)

options(repr.plot.width=10, repr.plot.height=3.5)
par(mfrow=c(1,3))
plot(fit, which=c(1,2,3))

#### Run F-test to determine if a cubic is necessary

In [ ]:
fit <- lm(log(number_active_physicians)~
          poly(income_percapita, 3) +        # orthogonal polynomial
          region +                           # 4 regions
          poly(income_percapita, 3):region   # interaction
          , data=cdi)

fit_reduced <- lm(log(number_active_physicians)~
          poly(income_percapita, 2) +        # orthogonal polynomial
          region +                           # 4 regions
          poly(income_percapita, 2):region   # interaction
          , data=cdi)


anova(fit_reduced, fit)

<font color='red'>**(2 points) Question:**</font> 
- What is the conclusion of the F-test? What does the result suggest?

#### Rerun regression using orthogonal quadratic functions

In [ ]:
fit <- lm(log(number_active_physicians)~
          poly(income_percapita, 2) +        # orthogonal polynomial
          region +                           # 4 regions
          poly(income_percapita, 2):region   # interaction
          , data=cdi)
summary(fit)

options(repr.plot.width=10, repr.plot.height=3.5)
par(mfrow=c(1,3))
plot(fit, which=c(1,2,3))

- It does not appear as if the polynomial representation of income_percapita interacts with region
- We can test this, again using the F-test

#### Run F-test to determine if there is an interaction between region and quadratic income_percapita

In [ ]:
fit <- lm(log(number_active_physicians)~
          poly(income_percapita, 2) +        # orthogonal polynomial
          region +                           # 4 regions
          poly(income_percapita, 2):region   # interaction
          , data=cdi)

fit_reduced <- lm(log(number_active_physicians)~
          poly(income_percapita, 2) +        # orthogonal polynomial
          region                             # 4 regions
          , data=cdi)

anova(fit_reduced, fit)

<font color='red'>**(2 points) Question:**</font> 
- What is the conclusion of the F-test? What does the result suggest?

#### Rerun regression without the interaction between region and income_percapita

- We are now narrowing in on our final regression model!

In [ ]:
fit <- lm(log(number_active_physicians)~
          poly(income_percapita, 2) +        # orthogonal polynomial
          region                             # 4 regions
          , data=cdi)
summary(fit)

options(repr.plot.width=10, repr.plot.height=3.5)
par(mfrow=c(1,3))
plot(fit, which=c(1,2,3))

### Conduct formal inference on parameters

In [ ]:
summary(glht(fit, linfct = mcp(region = "Tukey")))

In [ ]:
# contrasts by hand (more flexibility, but can be tedious to create)
contrast <- matrix(c(0, 0, 0, 1, 0, 0,
                     0, 0, 0, 0, 1, 0,
                     0, 0, 0, 0, 0, 1,
                     0, 0, 0, 1, -1, 0,
                     0, 0, 0, 1, 0, -1,
                     0, 0, 0, 0, 1, -1), nrow=6, byrow=TRUE)
summary(glht(fit, linfct = contrast))

### Create plots to analyze inferences

In [ ]:
# final model
fit <- lm(log(number_active_physicians)~
          poly(income_percapita, 2) +        # orthogonal polynomial
          region                             # 4 regions
          , data=cdi)

In [ ]:
# calculate predictions for regions
income_vector <- seq(min(cdi$income_percapita), max(cdi$income_percapita), 1000)

# northeast region
newdata <- data.frame(income_percapita = income_vector,
                      region = "NE")
pred_regionNE <- predict.lm(fit, newdata = newdata, interval="confidence")


# northcentral
newdata <- data.frame(income_percapita = income_vector,
                      region = "NC")
pred_regionNC <- predict.lm(fit, newdata = newdata, interval="confidence")


# southern
newdata <- data.frame(income_percapita = income_vector,
                      region = "S")
pred_regionS <- predict.lm(fit, newdata = newdata, interval="confidence")


# western
newdata <- data.frame(income_percapita = income_vector,
                      region = "W")
pred_regionW <- predict.lm(fit, newdata = newdata, interval="confidence")


# combine into a single dataframe
preds_nocorrection <- rbind(data.frame(pred_regionNE, income_percapita = income_vector, region="NE"),
                            data.frame(pred_regionNC, income_percapita = income_vector, region="NC"),
                            data.frame(pred_regionS, income_percapita = income_vector, region="S"),
                            data.frame(pred_regionW, income_percapita = income_vector, region="W"))

- Visualize the functional relationships

In [ ]:
# plot
options(repr.plot.width=8, repr.plot.height=5)
ggplot(cdi, aes(x=income_percapita, y=log(number_active_physicians), color=region)) +
  geom_point() + 
  geom_line(data = preds_nocorrection, aes(x=income_percapita, y=fit, color=region)) 

- Visualize without log scale

In [ ]:
# plot
options(repr.plot.width=8, repr.plot.height=5)
ggplot(cdi, aes(x=income_percapita, y=(number_active_physicians), color=region)) +
  geom_point() + 
  geom_line(data = preds_nocorrection, aes(x=income_percapita, y=exp(fit), color=region)) 

In [ ]:
# plot
options(repr.plot.width=8, repr.plot.height=5)
ggplot(cdi, aes(x=income_percapita, y=(number_active_physicians), color=region)) +
  #geom_point() + 
  geom_line(data = preds_nocorrection, aes(x=income_percapita, y=exp(fit), color=region))  
  #geom_line(data = preds_nocorrection, aes(x=income_percapita, y=exp(lwr), color=region), lty=2) + 
  #geom_line(data = preds_nocorrection, aes(x=income_percapita, y=exp(upr), color=region), lty=2) 

- Connect inferences with what you observe visually

In [ ]:
summary(fit)

summary(glht(fit, linfct = mcp(region = "Tukey")))

<font color='red'>**(2 points) Question:**</font> 
- What are your final conclusions? Write your conclusions in a paragraph.

### Data Analysis Objective II - If time in class

We are interested in the relationship between the number of active physicians (Y), income per capita ($X_{1}$), and total population ($X_{2}$) across the four regions ($X_{3}$) in the CDI dataset.

Perform a similar analysis as shown above.